# Day11_1: 텍스트 분석 기법 (Text Analysis Techniques) - 정답

## 📚 학습 목표

**Part 1: 기초**
1. 단어 빈도(Counter)로 텍스트 분석 시작하기
2. n-gram 개념 이해하고 활용하기
3. CountVectorizer로 문서-단어 행렬 만들기
4. TF-IDF 개념 이해하기
5. TfidfVectorizer로 키워드 중요도 추출하기

**Part 2: 심화**
1. 코사인 유사도로 문서 비교하기
2. 자카드 유사도 이해하기
3. 중복 문서 탐지하기
4. 간단한 문서 검색 시스템 구현하기

---

## 🎯 실습 퀴즈 정답

---

### Q1. 단어 빈도 계산 ⭐

**문제**: 다음 리뷰 텍스트에서 가장 많이 등장한 단어와 그 빈도를 출력하세요.

```python
review = "배송이 빠르고 상품이 좋아요 배송도 빠르고 서비스도 좋아요"
```

**기대 결과**: `('좋아요', 2)` 또는 `('빠르고', 2)`

In [ ]:
# Q1 정답
from collections import Counter

review = "배송이 빠르고 상품이 좋아요 배송도 빠르고 서비스도 좋아요"

# 1. 단어 분리
words = review.split()

# 2. 빈도 계산
word_counts = Counter(words)

# 3. 가장 많이 등장한 단어
most_common_word = word_counts.most_common(1)[0]

print(f"가장 많이 등장한 단어: {most_common_word}")

In [ ]:
# 테스트 케이스로 검증
assert word_counts.most_common(1)[0][1] == 2, "최다 빈도가 2여야 합니다"
assert most_common_word[0] in ['좋아요', '빠르고'], "가장 빈번한 단어는 '좋아요' 또는 '빠르고'여야 합니다"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`split()`으로 단어를 분리하고 `Counter`로 빈도를 계산합니다.

**핵심 개념**:
- `Counter`: 요소 빈도를 자동으로 세는 딕셔너리 서브클래스
- `most_common(n)`: 상위 n개 빈출 요소 반환

**대안 솔루션**:
```python
# 딕셔너리로 직접 구현
word_counts = {}
for word in words:
    word_counts[word] = word_counts.get(word, 0) + 1
```

**흔한 실수**:
- `most_common()` 결과가 리스트 안의 튜플임을 잊음
- 인덱싱 없이 `most_common(1)` 자체를 출력

**실무 팁**:
고객 리뷰에서 자주 언급되는 키워드를 빠르게 파악할 때 Counter를 활용합니다.

---

### Q2. bigram 추출 ⭐

**문제**: 주어진 텍스트에서 모든 bigram(연속된 2단어)을 추출하세요.

```python
text = "파이썬 데이터 분석 입문"
```

**기대 결과**: `['파이썬 데이터', '데이터 분석', '분석 입문']`

In [ ]:
# Q2 정답
text = "파이썬 데이터 분석 입문"

# 1. 단어 분리
words = text.split()

# 2. bigram 생성
bigrams = []
for i in range(len(words) - 1):  # n-1번 반복 (n=2)
    bigram = words[i] + ' ' + words[i + 1]
    bigrams.append(bigram)

print(f"Bigrams: {bigrams}")

In [ ]:
# 테스트 케이스로 검증
expected = ['파이썬 데이터', '데이터 분석', '분석 입문']
assert bigrams == expected, f"Expected {expected}, got {bigrams}"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
인덱스 i부터 i+1까지 슬라이딩하며 연속된 2단어를 추출합니다.

**핵심 개념**:
- n-gram: 연속된 n개 요소의 시퀀스
- 슬라이딩 윈도우: 고정 크기 창을 이동시키며 처리

**대안 솔루션**:
```python
# 리스트 컴프리헨션 사용
bigrams = [' '.join(words[i:i+2]) for i in range(len(words)-1)]

# zip 활용
bigrams = [f"{w1} {w2}" for w1, w2 in zip(words[:-1], words[1:])]
```

**흔한 실수**:
- `range(len(words))`로 하면 마지막에 인덱스 초과 에러
- 단어 사이 공백을 잊음

**실무 팁**:
bigram은 "배송이 빠르다", "맛있고 신선"처럼 표현 패턴을 찾을 때 유용합니다.

---

### Q3. CountVectorizer 기초 ⭐⭐

**문제**: CountVectorizer를 사용하여 문서-단어 행렬을 만들고, '배송'이라는 단어가 각 문서에 몇 번 등장하는지 출력하세요.

```python
documents = [
    "배송이 빨라요 배송 최고",
    "상품이 좋아요",
    "배송도 빠르고 상품도 좋아요"
]
```

In [ ]:
# Q3 정답
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

documents = [
    "배송이 빨라요 배송 최고",
    "상품이 좋아요",
    "배송도 빠르고 상품도 좋아요"
]

# 1. CountVectorizer 생성 및 변환
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(documents)

# 2. 단어 목록 확인
feature_names = vectorizer.get_feature_names_out()
print("단어 목록:", feature_names)

# 3. '배송' 단어의 인덱스 찾기
if '배송' in feature_names:
    idx = list(feature_names).index('배송')
    # 각 문서에서 '배송' 빈도
    for i, doc in enumerate(documents):
        count = count_matrix.toarray()[i][idx]
        print(f"문서{i+1}: '배송' = {count}회")
else:
    print("'배송' 단어가 없습니다 (형태소가 분리될 수 있음)")

In [ ]:
# 테스트 케이스로 검증
matrix_array = count_matrix.toarray()
assert matrix_array.shape[0] == 3, "문서 수가 3이어야 합니다"
print("\n문서-단어 행렬:")
df = pd.DataFrame(matrix_array, columns=feature_names)
print(df)
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
CountVectorizer로 문서를 벡터화한 후, 특정 단어의 인덱스를 찾아 빈도를 확인합니다.

**핵심 개념**:
- `fit_transform()`: 학습과 변환을 동시에 수행
- `get_feature_names_out()`: 단어 목록 반환
- `toarray()`: 희소 행렬을 밀집 배열로 변환

**대안 솔루션**:
```python
# DataFrame으로 직접 조회
df = pd.DataFrame(count_matrix.toarray(), columns=feature_names)
print(df['배송'] if '배송' in df.columns else "없음")
```

**흔한 실수**:
- `toarray()` 없이 희소 행렬 접근 시도
- 한글 형태소가 분리되어 원하는 단어가 없을 수 있음

**실무 팁**:
한글은 CountVectorizer 기본 설정으로 정확한 분석이 어려우므로, KoNLPy로 전처리 후 사용하는 것이 좋습니다.

---

### Q4. TF-IDF 키워드 추출 ⭐⭐

**문제**: TfidfVectorizer를 사용하여 첫 번째 문서의 TF-IDF 값이 가장 높은 단어를 찾으세요.

```python
documents = [
    "삼성전자 반도체 수출",
    "현대자동차 전기차 판매",
    "LG전자 가전 수출"
]
```

In [ ]:
# Q4 정답
from sklearn.feature_extraction.text import TfidfVectorizer

documents = [
    "삼성전자 반도체 수출",
    "현대자동차 전기차 판매",
    "LG전자 가전 수출"
]

# 1. TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(documents)

# 2. 단어 목록
feature_names = tfidf.get_feature_names_out()

# 3. 첫 번째 문서의 TF-IDF 값
first_doc_tfidf = tfidf_matrix[0].toarray().flatten()

# 4. 가장 높은 TF-IDF 단어 찾기
max_idx = first_doc_tfidf.argmax()
max_word = feature_names[max_idx]
max_score = first_doc_tfidf[max_idx]

print(f"첫 번째 문서의 최고 TF-IDF 단어: '{max_word}' (점수: {max_score:.4f})")

In [ ]:
# 테스트 케이스로 검증
# '반도체'와 '삼성전자'는 첫 번째 문서에만 등장하므로 높은 TF-IDF
assert max_word in ['반도체', '삼성전자'], f"Expected '반도체' or '삼성전자', got '{max_word}'"
print("\n모든 단어의 TF-IDF:")
for word, score in sorted(zip(feature_names, first_doc_tfidf), key=lambda x: -x[1]):
    if score > 0:
        print(f"  {word}: {score:.4f}")
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
TfidfVectorizer로 벡터화 후, argmax()로 최대 값의 인덱스를 찾습니다.

**핵심 개념**:
- TF-IDF: 문서 내 빈도(TF) x 희소성(IDF)
- 특정 문서에만 등장하는 단어 = 높은 IDF = 높은 TF-IDF

**대안 솔루션**:
```python
# pandas로 정렬
import pandas as pd
df = pd.DataFrame(tfidf_matrix[0].toarray(), columns=feature_names)
top_word = df.iloc[0].idxmax()
```

**흔한 실수**:
- `argmax()`가 인덱스를 반환하는 것을 잊음
- 희소 행렬을 밀집 배열로 변환하지 않음

**실무 팁**:
TF-IDF 상위 키워드는 문서 자동 태깅, 키워드 광고 최적화에 활용됩니다.

---

### Q5. 자카드 유사도 계산 ⭐⭐

**문제**: 두 문서의 자카드 유사도를 계산하세요.

```python
doc1 = "파이썬 데이터 분석"
doc2 = "파이썬 머신러닝 분석"
```

**힌트**: 교집합 / 합집합

In [ ]:
# Q5 정답
doc1 = "파이썬 데이터 분석"
doc2 = "파이썬 머신러닝 분석"

# 1. 단어 집합으로 변환
set1 = set(doc1.split())
set2 = set(doc2.split())

print(f"문서1 단어: {set1}")
print(f"문서2 단어: {set2}")

# 2. 교집합과 합집합
intersection = set1 & set2  # 또는 set1.intersection(set2)
union = set1 | set2         # 또는 set1.union(set2)

print(f"\n교집합: {intersection}")
print(f"합집합: {union}")

# 3. 자카드 유사도 계산
jaccard = len(intersection) / len(union)

print(f"\n자카드 유사도: {jaccard:.4f}")

In [ ]:
# 테스트 케이스로 검증
# 교집합: {'파이썬', '분석'} = 2개
# 합집합: {'파이썬', '데이터', '분석', '머신러닝'} = 4개
# 자카드 = 2/4 = 0.5
expected_jaccard = 2 / 4
assert abs(jaccard - expected_jaccard) < 0.001, f"Expected {expected_jaccard}, got {jaccard}"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
문자열을 집합으로 변환하고, 집합 연산으로 교집합과 합집합을 구합니다.

**핵심 개념**:
- 자카드 유사도 = |A intersection B| / |A union B|
- 집합 연산: `&` (교집합), `|` (합집합)

**대안 솔루션**:
```python
# 메서드 사용
intersection = set1.intersection(set2)
union = set1.union(set2)
```

**흔한 실수**:
- 리스트로 계산하면 중복이 포함됨 (집합 변환 필수)
- 교집합과 합집합을 혼동

**실무 팁**:
자카드 유사도는 해시태그 비교, 짧은 텍스트 유사도 측정에 적합합니다.

---

## Part 2 심화 퀴즈 정답

---

### Q6. 코사인 유사도로 유사 문서 찾기 ⭐⭐⭐

**문제**: 기준 문서와 가장 유사한 문서를 찾고 유사도를 출력하세요.

```python
documents = [
    "파이썬 머신러닝 딥러닝",  # 기준 문서 (인덱스 0)
    "자바 웹 개발",
    "파이썬 데이터 분석 머신러닝",
    "자바스크립트 프론트엔드"
]
```

**기대 결과**: 문서 인덱스와 유사도 값

In [ ]:
# Q6 정답
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

documents = [
    "파이썬 머신러닝 딥러닝",
    "자바 웹 개발",
    "파이썬 데이터 분석 머신러닝",
    "자바스크립트 프론트엔드"
]

# 1. TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(documents)

# 2. 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix)

# 3. 기준 문서(인덱스 0)와 다른 문서들의 유사도
target_idx = 0
similarities = list(enumerate(cosine_sim[target_idx]))

# 4. 자기 자신 제외하고 정렬
similarities = [(i, sim) for i, sim in similarities if i != target_idx]
similarities.sort(key=lambda x: x[1], reverse=True)

# 5. 가장 유사한 문서
most_similar_idx, most_similar_score = similarities[0]

print(f"기준 문서: '{documents[target_idx]}'")
print(f"\n가장 유사한 문서:")
print(f"  인덱스: {most_similar_idx}")
print(f"  내용: '{documents[most_similar_idx]}'")
print(f"  유사도: {most_similar_score:.4f}")

In [ ]:
# 테스트 케이스로 검증
# 문서0과 문서2는 '파이썬', '머신러닝'을 공유 -> 가장 유사
assert most_similar_idx == 2, f"Expected index 2, got {most_similar_idx}"
assert most_similar_score > 0.3, "유사도가 0.3 이상이어야 합니다"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
TF-IDF로 벡터화 -> 코사인 유사도 계산 -> 자기 자신 제외하고 최대값 찾기

**핵심 개념**:
- `cosine_similarity()`: 모든 문서 쌍의 유사도 행렬 반환
- 대각선은 자기 자신(1.0)이므로 제외

**대안 솔루션**:
```python
# numpy argsort 사용
import numpy as np
sim_scores = cosine_sim[0].copy()
sim_scores[0] = -1  # 자기 자신 제외
most_similar = np.argmax(sim_scores)
```

**흔한 실수**:
- 자기 자신을 제외하지 않으면 항상 자신이 가장 유사
- 인덱스와 유사도 값을 혼동

**실무 팁**:
추천 시스템에서 "이 기사를 읽은 사람들이 좋아한 기사"를 구현할 때 사용합니다.

---

### Q7. n-gram + TF-IDF 조합 ⭐⭐⭐

**문제**: unigram과 bigram을 모두 포함하는 TfidfVectorizer를 만들고, 첫 번째 문서에서 TF-IDF 값이 가장 높은 상위 3개 feature를 출력하세요.

```python
documents = [
    "배송이 빠르고 상품 품질 좋아요",
    "상품 품질은 좋은데 배송이 느려요",
    "가격 대비 만족스러워요"
]
```

In [ ]:
# Q7 정답
from sklearn.feature_extraction.text import TfidfVectorizer

documents = [
    "배송이 빠르고 상품 품질 좋아요",
    "상품 품질은 좋은데 배송이 느려요",
    "가격 대비 만족스러워요"
]

# 1. unigram + bigram TF-IDF 벡터화
tfidf = TfidfVectorizer(ngram_range=(1, 2))  # (1, 2) = unigram + bigram
tfidf_matrix = tfidf.fit_transform(documents)

# 2. feature 이름
feature_names = tfidf.get_feature_names_out()

# 3. 첫 번째 문서의 TF-IDF 값
first_doc_tfidf = tfidf_matrix[0].toarray().flatten()

# 4. 단어와 점수를 묶어서 정렬
word_scores = list(zip(feature_names, first_doc_tfidf))
word_scores_sorted = sorted(word_scores, key=lambda x: x[1], reverse=True)

# 5. 상위 3개 출력
print("첫 번째 문서의 상위 3개 feature:")
for word, score in word_scores_sorted[:3]:
    print(f"  '{word}': {score:.4f}")

In [ ]:
# 테스트 케이스로 검증
# 상위 3개 모두 TF-IDF > 0 이어야 함
top_3 = word_scores_sorted[:3]
assert all(score > 0 for _, score in top_3), "상위 3개 TF-IDF는 0보다 커야 합니다"

# bigram도 포함되어 있는지 확인
has_bigram = any(' ' in word for word, _ in word_scores_sorted[:10] if word)
print(f"\n전체 feature 수: {len(feature_names)}")
print(f"bigram 포함 여부: {has_bigram}")
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`ngram_range=(1, 2)` 옵션으로 unigram과 bigram을 모두 생성하고, 정렬하여 상위 키워드를 추출합니다.

**핵심 개념**:
- `ngram_range`: (min_n, max_n) 형태로 n-gram 범위 지정
- bigram은 "배송이 빠르고"처럼 연속 2단어

**대안 솔루션**:
```python
# DataFrame으로 정렬
import pandas as pd
df = pd.DataFrame({'word': feature_names, 'tfidf': first_doc_tfidf})
top_3 = df.nlargest(3, 'tfidf')
```

**흔한 실수**:
- `ngram_range`를 리스트 `[1, 2]`로 쓰면 에러 (튜플이어야 함)
- `(2, 2)`로 하면 bigram만 생성

**실무 팁**:
리뷰 분석에서 bigram을 포함하면 "배송 느림", "품질 좋음" 같은 의미 있는 표현을 잡을 수 있습니다.

---

### Q8. 중복 문서 탐지 ⭐⭐⭐⭐

**문제**: 코사인 유사도가 0.6 이상인 문서 쌍을 모두 찾아 출력하세요.

```python
documents = [
    "삼성전자 반도체 수출 증가",
    "현대차 전기차 판매 호조",
    "삼성 반도체 수출 사상 최대",  # 0번과 유사
    "LG전자 가전 사업 성장",
    "삼성전자 반도체 수출 신기록"   # 0번과 유사
]
```

In [ ]:
# Q8 정답
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

documents = [
    "삼성전자 반도체 수출 증가",
    "현대차 전기차 판매 호조",
    "삼성 반도체 수출 사상 최대",
    "LG전자 가전 사업 성장",
    "삼성전자 반도체 수출 신기록"
]

# 1. TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(documents)

# 2. 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix)

# 3. 임계값 이상인 쌍 찾기
threshold = 0.6
duplicates = []

n = len(documents)
for i in range(n):
    for j in range(i + 1, n):  # 상삼각 행렬만 확인 (중복 방지)
        if cosine_sim[i][j] >= threshold:
            duplicates.append({
                'idx1': i,
                'idx2': j,
                'doc1': documents[i],
                'doc2': documents[j],
                'similarity': round(cosine_sim[i][j], 4)
            })

# 4. 결과 출력
print(f"=== 유사도 {threshold} 이상인 문서 쌍 ===")
for dup in duplicates:
    print(f"\n유사도: {dup['similarity']}")
    print(f"  문서{dup['idx1']}: {dup['doc1']}")
    print(f"  문서{dup['idx2']}: {dup['doc2']}")

In [ ]:
# 테스트 케이스로 검증
# 문서 0, 2, 4가 서로 유사해야 함
expected_pairs = {(0, 2), (0, 4), (2, 4)}
found_pairs = {(d['idx1'], d['idx2']) for d in duplicates}

print(f"\n발견된 쌍: {found_pairs}")
assert len(duplicates) >= 2, "최소 2개 이상의 유사 쌍이 있어야 합니다"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
모든 문서 쌍의 유사도를 계산하고, 임계값 이상인 쌍만 필터링합니다.

**핵심 개념**:
- 상삼각 행렬: i < j인 쌍만 확인하여 중복 방지
- 임계값(threshold): 비즈니스 요구에 따라 조정

**대안 솔루션**:
```python
# numpy where 사용
import numpy as np
pairs = np.where((cosine_sim >= threshold) & (np.triu(np.ones_like(cosine_sim), k=1).astype(bool)))
```

**흔한 실수**:
- 전체 행렬을 순회하면 (i, j)와 (j, i)가 중복
- 대각선(자기 자신)을 제외하지 않음

**실무 팁**:
뉴스 중복 기사 탐지, 표절 검사에 활용됩니다. 임계값은 보통 0.7~0.9 사이로 설정합니다.

---

### Q9. 검색 쿼리 매칭 ⭐⭐⭐⭐

**문제**: 검색 쿼리 "데이터 분석"과 가장 관련 높은 문서 2개를 찾아 유사도와 함께 출력하세요.

```python
documents = [
    "파이썬 기초 문법 강좌",
    "데이터 분석을 위한 판다스",
    "웹 개발 입문 가이드",
    "머신러닝 데이터 전처리",
    "SQL 데이터베이스 기초"
]

query = "데이터 분석"
```

In [ ]:
# Q9 정답
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

documents = [
    "파이썬 기초 문법 강좌",
    "데이터 분석을 위한 판다스",
    "웹 개발 입문 가이드",
    "머신러닝 데이터 전처리",
    "SQL 데이터베이스 기초"
]

query = "데이터 분석"

# 1. 문서로 TF-IDF 학습
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(documents)

# 2. 쿼리를 벡터로 변환 (기존 vocabulary 사용)
query_vector = tfidf.transform([query])

# 3. 쿼리와 모든 문서의 유사도 계산
similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

# 4. 상위 2개 문서 찾기
top_indices = similarities.argsort()[::-1][:2]  # 내림차순 정렬 후 상위 2개

print(f"검색 쿼리: '{query}'")
print("\n검색 결과 (상위 2개):")
for rank, idx in enumerate(top_indices, 1):
    print(f"  {rank}. [{similarities[idx]:.4f}] {documents[idx]}")

In [ ]:
# 테스트 케이스로 검증
# "데이터 분석"이 포함된 문서1이 1위여야 함
assert top_indices[0] == 1, f"Expected index 1, got {top_indices[0]}"
assert similarities[top_indices[0]] > similarities[top_indices[1]], "1위가 2위보다 유사도 높아야 함"
print("\n테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
문서로 학습된 TF-IDF 벡터라이저를 사용하여 쿼리를 변환하고, 모든 문서와 유사도를 계산합니다.

**핵심 개념**:
- `fit_transform(documents)`: 문서로 학습 + 변환
- `transform([query])`: 기존 vocabulary로 쿼리 변환
- 쿼리에 없는 단어는 0으로 처리됨

**대안 솔루션**:
```python
# numpy argpartition (대용량에 더 효율적)
import numpy as np
top_2 = np.argpartition(similarities, -2)[-2:]
top_2_sorted = top_2[np.argsort(similarities[top_2])[::-1]]
```

**흔한 실수**:
- `fit_transform()`을 쿼리에도 사용하면 vocabulary가 달라짐
- `argsort()`가 오름차순임을 잊고 그대로 사용

**실무 팁**:
검색 엔진 구현 시 쿼리 확장(query expansion)을 추가하면 정확도가 향상됩니다.

---

### Q10. 종합: 문서 분류 및 키워드 추출 시스템 ⭐⭐⭐⭐⭐

**문제**: 다음 요구사항을 모두 만족하는 분석 시스템을 구현하세요.

1. TF-IDF로 각 문서의 상위 2개 키워드 추출
2. 코사인 유사도로 가장 유사한 문서 쌍 찾기
3. Plotly로 문서 간 유사도 히트맵 시각화

```python
documents = [
    "삼성전자 스마트폰 갤럭시 신제품 출시",
    "애플 아이폰 신제품 발표",
    "현대차 전기차 아이오닉 판매 증가",
    "테슬라 전기차 모델Y 인기",
    "삼성 갤럭시 폴드 스마트폰 혁신"
]
```

In [ ]:
# Q10 정답
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import plotly.figure_factory as ff
import pandas as pd

documents = [
    "삼성전자 스마트폰 갤럭시 신제품 출시",
    "애플 아이폰 신제품 발표",
    "현대차 전기차 아이오닉 판매 증가",
    "테슬라 전기차 모델Y 인기",
    "삼성 갤럭시 폴드 스마트폰 혁신"
]

# === 1. TF-IDF로 각 문서의 상위 2개 키워드 추출 ===
print("=== 1. 문서별 상위 2개 키워드 ===")

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(documents)
feature_names = tfidf.get_feature_names_out()

for i, doc in enumerate(documents):
    # 해당 문서의 TF-IDF 점수
    tfidf_scores = tfidf_matrix[i].toarray().flatten()
    
    # 상위 2개 인덱스
    top_indices = tfidf_scores.argsort()[::-1][:2]
    
    # 키워드 추출
    top_keywords = [(feature_names[idx], round(tfidf_scores[idx], 3)) for idx in top_indices]
    
    print(f"문서{i}: {top_keywords}")

In [ ]:
# === 2. 가장 유사한 문서 쌍 찾기 ===
print("\n=== 2. 가장 유사한 문서 쌍 ===")

cosine_sim = cosine_similarity(tfidf_matrix)

# 자기 자신 제외하고 최대 유사도 찾기
max_sim = 0
max_pair = (0, 0)

n = len(documents)
for i in range(n):
    for j in range(i + 1, n):
        if cosine_sim[i][j] > max_sim:
            max_sim = cosine_sim[i][j]
            max_pair = (i, j)

print(f"유사도: {max_sim:.4f}")
print(f"문서{max_pair[0]}: {documents[max_pair[0]]}")
print(f"문서{max_pair[1]}: {documents[max_pair[1]]}")

In [ ]:
# === 3. Plotly 히트맵 시각화 ===
print("\n=== 3. 유사도 히트맵 ===")

# 문서 라벨 (짧게 줄임)
labels = [f"문서{i}" for i in range(len(documents))]

# 히트맵 생성
fig = ff.create_annotated_heatmap(
    z=cosine_sim.round(2),
    x=labels,
    y=labels,
    colorscale='Blues',
    showscale=True
)

fig.update_layout(
    title='문서 간 코사인 유사도 히트맵',
    xaxis_title='문서',
    yaxis_title='문서',
    width=600,
    height=500
)

fig.show()

In [ ]:
# 테스트 케이스로 검증
# 문서0과 문서4 (삼성, 갤럭시, 스마트폰) 또는 문서2와 문서3 (전기차)가 유사해야 함
assert max_sim > 0.2, "최대 유사도가 0.2 이상이어야 합니다"
assert max_pair in [(0, 4), (2, 3)], f"예상 쌍: (0, 4) 또는 (2, 3), 실제: {max_pair}"
print("\n모든 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. TF-IDF 벡터화로 각 문서의 단어 중요도 계산
2. 코사인 유사도 행렬에서 최대값 찾기
3. Plotly annotated heatmap으로 시각화

**핵심 개념**:
- 문서별 키워드: TF-IDF 상위 값의 단어
- 유사 문서: 코사인 유사도 최대 쌍
- 히트맵: 행렬 데이터를 색상으로 표현

**대안 솔루션**:
```python
# pandas + seaborn 스타일
import plotly.express as px
df_sim = pd.DataFrame(cosine_sim, index=labels, columns=labels)
fig = px.imshow(df_sim, text_auto='.2f', color_continuous_scale='Blues')
```

**흔한 실수**:
- 히트맵 라벨 순서를 잘못 지정
- 색상 스케일 방향이 반대 (낮은 값이 진한 색)

**실무 팁**:
이 시스템은 뉴스 클러스터링, 문서 추천, 중복 탐지의 기반이 됩니다. 실무에서는 전처리(형태소 분석, 불용어 제거)를 추가합니다.

---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/클래스 | 실무 활용 |
|------|------------------|----------|
| 단어 빈도 | `Counter`, `most_common()` | 키워드 순위, 불만 분석 |
| n-gram | 직접 구현 또는 `ngram_range` | 연속 표현 패턴 탐지 |
| CountVectorizer | `fit_transform()`, `get_feature_names_out()` | 문서-단어 행렬 생성 |
| TF-IDF | `TfidfVectorizer` | 핵심 키워드 추출, 중요도 측정 |

### Part 2: 심화 핵심 요약

| 기법 | 사용법 | 언제 쓰나? |
|------|--------|----------|
| 코사인 유사도 | `cosine_similarity(tfidf_matrix)` | 긴 문서 비교, 추천 시스템 |
| 자카드 유사도 | `len(A&B) / len(A|B)` | 짧은 텍스트, 태그 비교 |
| 중복 탐지 | 유사도 > 임계값 | 표절 검사, 스팸 필터링 |
| 문서 검색 | 쿼리 벡터 + 유사도 | FAQ 봇, 검색 엔진 |

### 💡 실무 팁

1. **전처리 중요**: TF-IDF 전에 불용어 제거, 형태소 분석 적용
2. **n-gram 범위**: bigram까지가 일반적, trigram은 데이터 많을 때만
3. **유사도 임계값**: 중복 탐지 시 0.7~0.8, 추천은 0.3~0.5
4. **희소 행렬**: 대용량 문서는 `sparse matrix` 유지 (toarray() 자제)
5. **한글 분석**: KoNLPy 형태소 분석과 조합하면 정확도 향상